# Cuda Study

need the execution of the file src/compilation.ipynb

## definition of the version
V1 : not recalculate the metadata if their aren't mutation
V2 : recalculate the metadata at once.


## get the results


In [1]:
import os
import subprocess

import experimentsDef as expDef

import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [5]:
implementation = expDef.getImplementationDetails()

experimentsHandDetails = expDef.getExpDetails()

relevantCodeVersion = ["baseline", "CUDAV1", "CUDAV2"]

experimentsRelevantDetails = {
    "perDefaultCuda" : {
        "w" : 32,
        "h" : 32,
        "g" : 5000,
        "m" : 0.00001,
    },
    "chargeGPU" : {
        "w" : 128,
        "h" : 128,
        "g" : 5000,
        "m" : 0.00001,
    },
    "chargeMoreRateGPU" : {
        "w" : 128,
        "h" : 128,
        "g" : 5000,
        "m" : 0.001,
    },
    "chargeLessRateGPU" : {
        "w" : 128,
        "h" : 128,
        "g" : 5000,
        "m" : 0.000001,
    }
}

columnsCSV = ["Gen","Resource","Stamp","TimeStamp_Start","TimeStamp_End","Duration","Indiv_id", "exec", "w", "h", "g", "mutation_rate", "versionCode", "versionExp"]


In [6]:
aggregate = pd.DataFrame(data=None, index=None, columns=columnsCSV)
workDir = os.getcwd()

for version in relevantCodeVersion:
    for exp in experimentsRelevantDetails:
        path = os.path.join(workDir, "simulations", version, exp, "trace.csv")
        data = pd.read_csv(path, sep=',')
        data["exec"] = "nameCPU"
        data["w"] = experimentsRelevantDetails[exp]["w"]
        data["h"] = experimentsRelevantDetails[exp]["h"]
        data["g"] = experimentsRelevantDetails[exp]["g"]
        data["mutation_rate"] = experimentsRelevantDetails[exp]["m"]
        data["versionCode"] = version
        data["versionExp"] = exp
        aggregate = aggregate.append(data, ignore_index=True)


## compare the results

In [7]:
df = aggregate[aggregate['Stamp'] == "STEP"]
df["Duration"] = df["Duration"].astype(float)
dfExp = df.groupby(["versionCode", "versionExp"]).mean().reset_index()
print(dfExp)

   versionCode         versionExp      Duration  mutation_rate
0       CUDAV1          chargeGPU  1.980742e+07       0.000010
1       CUDAV1  chargeLessRateGPU  1.326312e+07       0.000001
2       CUDAV1  chargeMoreRateGPU  1.087049e+08       0.001000
3       CUDAV1     perDefaultCuda  4.448774e+06       0.000010
4       CUDAV2          chargeGPU  7.641949e+06       0.000010
5       CUDAV2  chargeLessRateGPU  5.068220e+06       0.000001
6       CUDAV2  chargeMoreRateGPU  5.937282e+07       0.001000
7       CUDAV2     perDefaultCuda  1.800791e+06       0.000010
8     baseline          chargeGPU  1.013746e+08       0.000010
9     baseline  chargeLessRateGPU  1.197790e+08       0.000001
10    baseline  chargeMoreRateGPU  1.261957e+08       0.001000
11    baseline     perDefaultCuda  9.681058e+06       0.000010


In [8]:
# get the ratio with baseline for each experiment

def getDurationRef(df, versionExp):
    dfIn = df[(df["versionExp"] == versionExp) & (df["versionCode"] == "baseline")]
    return dfIn["Duration"].values[0]
    

dfExp["ratio"] = dfExp.apply(lambda row : getDurationRef(dfExp, row["versionExp"])/row["Duration"], axis=1)

print(dfExp)

   versionCode         versionExp      Duration  mutation_rate      ratio
0       CUDAV1          chargeGPU  1.980742e+07       0.000010   5.118014
1       CUDAV1  chargeLessRateGPU  1.326312e+07       0.000001   9.030986
2       CUDAV1  chargeMoreRateGPU  1.087049e+08       0.001000   1.160901
3       CUDAV1     perDefaultCuda  4.448774e+06       0.000010   2.176118
4       CUDAV2          chargeGPU  7.641949e+06       0.000010  13.265548
5       CUDAV2  chargeLessRateGPU  5.068220e+06       0.000001  23.633349
6       CUDAV2  chargeMoreRateGPU  5.937282e+07       0.001000   2.125478
7       CUDAV2     perDefaultCuda  1.800791e+06       0.000010   5.376003
8     baseline          chargeGPU  1.013746e+08       0.000010   1.000000
9     baseline  chargeLessRateGPU  1.197790e+08       0.000001   1.000000
10    baseline  chargeMoreRateGPU  1.261957e+08       0.001000   1.000000
11    baseline     perDefaultCuda  9.681058e+06       0.000010   1.000000


On peux voir qu'on a gagné (V2 est 4 fois plus rapide que le code de base) en temps d'execution, et plus on augmente le taux de mutation, plus on gagne en temps d'execution.